<a href="https://colab.research.google.com/github/SuveekshaVullendula/alzheimers/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import datetime
import smtplib
from email.mime.text import MIMEText

In [ ]:
def send_notification(message, recipient_email):
    print(f"[NOTIFICATION]: {message}")
    #Uncomment below for real email sending
    msg = MIMEText(message)
    msg['Subject'] = 'Alzheimers Patient Alert'
    msg['From'] = 'suveeksha287@gmail.com'
    msg['To'] = recipient_email
    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()
    server.login('suveeksha287@gmail.com', 'mazl hwol acbw bith')
    server.sendmail('suveeksha287@gmail.com', recipient_email, msg.as_string())
    server.quit()

In [ ]:
df = pd.read_csv("dataset.csv")

In [ ]:
if 'LastSeenTime' in df.columns:
    # If it exists, proceed as before
    df['LastSeenHour'] = pd.to_datetime(df['LastSeenTime'], errors='coerce').dt.hour
else:
    # If not, create a 'LastSeenHour' column with random values between 0 and 23
    df['LastSeenHour'] = np.random.randint(0, 24, size=len(df))

In [ ]:
def categorize_time_of_day(hour):
    if 6 <= hour < 12:
        return 0  # Morning
    elif 12 <= hour < 18:
        return 1  # Afternoon
    elif 18 <= hour < 24:
        return 2  # Evening
    else:
        return 3  # Night

In [ ]:
df['TimeOfDay'] = df['LastSeenHour'].apply(lambda x: categorize_time_of_day(x) if pd.notnull(x) else np.random.randint(0, 4))

In [ ]:
# Encoding categorical variables
df['Gender'] = LabelEncoder().fit_transform(df['Gender'])
df['Education'] = LabelEncoder().fit_transform(df['EducationLevel'])

In [ ]:
# Feature Selection
features = ['Age', 'Gender', 'EducationLevel', 'SleepQuality', 'Depression', 'TimeOfDay',
            'PhysicalActivity', 'MemoryComplaints', 'BehavioralProblems', 'ADL', 'Confusion', 'CholesterolHDL']
target = 'RiskLevel'

In [ ]:
def determine_risk(radius):
    if radius < 500:
        return 0  # Low risk
    elif 500 <= radius < 1000:
        return 1  # Medium risk
    else:
        return 2  # High risk

df['RiskLevel'] = df['Radius'].apply(determine_risk)

X = df[features]
y = df[target]

In [ ]:
# Handle missing values (fill with median)
X.fillna(X.median(), inplace=True)

<ipython-input-11-9da93ea191d0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.fillna(X.median(), inplace=True)


In [ ]:
# Standardize features
scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

In [ ]:
# Handle class imbalance
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

In [ ]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

In [ ]:
# Step 4: Train Model
model = RandomForestClassifier(n_estimators=300, max_depth=15, random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(max_depth=15, n_estimators=300, random_state=42)

In [ ]:
# Step 5: Make Predictions
y_pred = model.predict(X_test)

In [ ]:
# Step 6: Evaluate Model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.8
Classification Report:
               precision    recall  f1-score   support

           0       0.76      0.84      0.80       286
           1       0.79      0.59      0.67       270
           2       0.84      0.98      0.91       269

    accuracy                           0.80       825
   macro avg       0.80      0.80      0.79       825
weighted avg       0.80      0.80      0.79       825



In [ ]:
# Step 7: Feature Importance
feature_importance = pd.Series(model.feature_importances_, index=features)
print("\nFeature Importance:\n", feature_importance.sort_values(ascending=False))


Feature Importance:
 ADL                   0.292458
Confusion             0.101655
PhysicalActivity      0.092950
SleepQuality          0.089696
CholesterolHDL        0.089389
EducationLevel        0.082287
Age                   0.079810
TimeOfDay             0.070314
BehavioralProblems    0.045639
Gender                0.021595
MemoryComplaints      0.019652
Depression            0.014556
dtype: float64


In [ ]:
# Step 8: Event-Based Monitoring & Notification Logic
notified_patients = set()  # To ensure notifications are sent only once per day
def process_new_patient(new_patient_data, recipient_email):
    patient_id = new_patient_data['PatientID']
    today = datetime.date.today()
    # Ensure notification is sent only once per day
    if (patient_id, today) in notified_patients:
        return
    new_patient = pd.DataFrame([new_patient_data], columns=features)
    new_patient = pd.DataFrame(scaler.transform(new_patient), columns=new_patient.columns)
    prediction = model.predict(new_patient)[0]
    if prediction == 2:  # High risk
        notify_time = max(0, new_patient_data['LastSeenHour'] - 1)
        send_notification(f"High Alert: Patient is very likely to cross at {new_patient_data['LastSeenHour']}:00. Notifying family at {notify_time}:40.", recipient_email)
    elif prediction == 1:  # Medium risk
        notify_time = max(0, new_patient_data['LastSeenHour'] - 2)
        send_notification(f"Warning: Patient may cross at {new_patient_data['LastSeenHour']}:00. Notifying family at {notify_time}:20.", recipient_email)
    notified_patients.add((patient_id, today))
# Simulated incoming new patient data (Replace with real-time data ingestion logic)
new_patient_data = {
    'PatientID': 101,
    'Age': 75, 'Gender': 1, 'Education': 2, 'SleepQuality': 3, 'Depression': 1,
    'TimeOfDay': 1, 'PhysicalActivity': 2, 'MemoryComplaints': 1, 'BehavioralProblems': 0,
    'ADL': 2, 'Confusion': 1, 'CholesterolHDL': 50, 'LastSeenHour': 7
}
recipient_email = "suveekshavullendula@gmail.com"
process_new_patient(new_patient_data, recipient_email)

In [ ]:
def process_new_patient(new_patient_data, recipient_email):
    patient_id = new_patient_data['PatientID']
    today = datetime.date.today()

    # Ensure the same notification is not sent multiple times a day
    if (patient_id, today) in notified_patients:
        return

    # Predict if the patient is likely to leave
    is_likely_to_go_out = determine_risk(new_patient_data)

    if is_likely_to_go_out == 1:
        alert_message = f"🚨 ALERT: Patient {patient_id} is likely to go outside at {new_patient_data['LastSeenHour']}:00. Please check on them!"
        send_notification(alert_message, recipient_email)

    # Mark this patient as notified for today
    notified_patients.add((patient_id, today))


In [ ]:
def determine_risk(patient_data):
    """
    Predicts if the patient is likely to go out based on behavioral and health factors.
    Returns 1 if they are likely to go out, otherwise 0.
    """
    new_patient = pd.DataFrame([patient_data], columns=features)
    new_patient = pd.DataFrame(scaler.transform(new_patient), columns=new_patient.columns)
    prediction = model.predict(new_patient)[0]
    return prediction  # 1 = Likely to go out, 0 = Unlikely to go out


In [ ]:
pip install schedule